# EDA and Data Cleaning Notebook

This notebook performs comprehensive Exploratory Data Analysis (EDA) and data cleaning on the computer dataset.

## Objectives:
1. Load and inspect raw data
2. Perform full EDA (shape, dtypes, missing values, distributions)
3. Identify data quality issues
4. Apply cleaning transformations
5. Export cleaned dataset


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings

warnings.filterwarnings('ignore')

# Add src to path - get the project root (parent of notebooks directory)
# Method 1: Try to get notebook directory from current working directory
current_dir = os.getcwd()

# Check if we're in the notebooks directory
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
elif 'notebooks' in current_dir:
    # If notebooks is in the path, go up to project root
    project_root = current_dir.split('notebooks')[0].rstrip(os.sep)
else:
    # Assume we're in project root
    project_root = current_dir

# Alternative method: Try to find project root by looking for src/cleaning directory
if not os.path.exists(os.path.join(project_root, 'src', 'cleaning')):
    # Try going up one level from current directory
    parent_dir = os.path.dirname(current_dir)
    if os.path.exists(os.path.join(parent_dir, 'src', 'cleaning')):
        project_root = parent_dir
    # Try current directory itself
    elif os.path.exists(os.path.join(current_dir, 'src', 'cleaning')):
        project_root = current_dir

src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Verify the path exists
if not os.path.exists(src_path):
    print(f"WARNING: src path not found at {src_path}")
    print(f"Current directory: {current_dir}")
    print(f"Project root: {project_root}")
    print("Please make sure you're running the notebook from the correct location.")
else:
    print(f"✓ Found src directory at: {src_path}")

# Import cleaning utilities
try:
    from cleaning import (
        extract_numeric_series,
        clean_storage_fields,
        clean_screen_fields,
        create_cleaned_cpu_gpu_columns,
        clean_multilabel_series,
        identify_multilabel_columns,
        clean_dataframe,
        get_default_config
    )
    print("✓ Cleaning modules imported successfully!")
except ImportError as e:
    print(f"✗ Error importing cleaning modules: {e}")
    print(f"\nTroubleshooting:")
    print(f"1. Make sure src/cleaning/ directory exists at: {src_path}")
    print(f"2. Check that all .py files are in src/cleaning/")
    print(f"3. Verify __init__.py exists in src/cleaning/")
    print(f"\nCurrent paths:")
    print(f"  - Current directory: {current_dir}")
    print(f"  - Project root: {project_root}")
    print(f"  - Source path: {src_path}")
    raise

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(f"\nProject root: {project_root}")
print(f"Source path: {src_path}")
print("Libraries imported successfully!")


ModuleNotFoundError: No module named 'cleaning'

## 1. Load Data


In [ ]:
# Load raw data
data_path = '../data/'
df_computers = pd.read_csv(f'{data_path}db_computers_2025_raw.csv')
df_cpu = pd.read_csv(f'{data_path}db_cpu_raw.csv')
df_gpu = pd.read_csv(f'{data_path}db_gpu_raw.csv')

print("=" * 80)
print("DATA LOADED")
print("=" * 80)
print(f"\nComputers dataset shape: {df_computers.shape}")
print(f"CPU dataset shape: {df_cpu.shape}")
print(f"GPU dataset shape: {df_gpu.shape}")
print(f"\nComputers columns: {len(df_computers.columns)}")


## 2. Initial Data Inspection


In [ ]:
# Basic info
print("=" * 80)
print("2.1 Dataset Shape and Types")
print("=" * 80)
print(f"Rows: {df_computers.shape[0]}")
print(f"Columns: {df_computers.shape[1]}")
print(f"\nData types:\n{df_computers.dtypes.value_counts()}")

# Display first few rows
print("\n" + "=" * 80)
print("First 3 rows:")
print("=" * 80)
df_computers.head(3)


In [ ]:
# Missing values analysis
print("=" * 80)
print("2.2 Missing Values Analysis")
print("=" * 80)
missing = df_computers.isnull().sum()
missing_pct = (missing / len(df_computers)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing Count', ascending=False)

print(f"\nColumns with missing values: {(missing > 0).sum()}")
print(f"\nTop 20 columns with most missing values:")
print(missing_df[missing_df['Missing Count'] > 0].head(20))


In [ ]:
# Duplicates
print("=" * 80)
print("2.3 Duplicate Rows")
print("=" * 80)
duplicates = df_computers.duplicated().sum()
print(f"Duplicate rows: {duplicates}")
print(f"Percentage: {(duplicates / len(df_computers)) * 100:.2f}%")


## 3. EDA - Missing Values Heatmap


In [ ]:
# Create missing values heatmap (sample of columns for visibility)
# Select columns with most missing values for visualization
top_missing_cols = missing_df[missing_df['Missing Count'] > 0].head(30).index.tolist()

if len(top_missing_cols) > 0:
    plt.figure(figsize=(16, 8))
    missing_data = df_computers[top_missing_cols].isnull()
    sns.heatmap(missing_data, yticklabels=False, cbar=True, cmap='viridis')
    plt.title('Missing Values Heatmap (Top 30 Columns)', fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('../docs/missing_values_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("No missing values to visualize")


## 4. EDA - Numeric Fields Analysis


In [ ]:
# Identify numeric fields stored as strings
print("=" * 80)
print("4.1 Numeric Fields Stored as Strings")
print("=" * 80)

numeric_candidates = [
    'Pantalla_Tamaño de la pantalla',
    'Pantalla_Diagonal de la pantalla',
    'RAM_Memoria RAM',
    'Disco duro_Capacidad de memoria SSD',
    'Disco duro_Capacidad del disco duro',
    'Procesador_Frecuencia de reloj',
    'Medidas y peso_Peso',
    'Alimentación_Vatios-hora',
]

print("\nSample values from numeric candidate columns:")
for col in numeric_candidates:
    if col in df_computers.columns:
        sample = df_computers[col].dropna().head(3)
        if len(sample) > 0:
            print(f"\n{col}:")
            for val in sample:
                print(f"  - {val}")


## 5. EDA - Categorical Fields Analysis


In [ ]:
# Value counts for key categorical fields
print("=" * 80)
print("5.1 Value Counts for Key Categorical Fields")
print("=" * 80)

categorical_cols = [
    'Procesador_Procesador',
    'Gráfica_Tarjeta gráfica',
    'Sistema operativo_Sistema operativo',
    'Serie',
    'Tipo de producto',
]

for col in categorical_cols:
    if col in df_computers.columns:
        print(f"\n{col}:")
        print(f"  Unique values: {df_computers[col].nunique()}")
        print(f"  Top 10 values:")
        print(df_computers[col].value_counts().head(10))
        print("-" * 80)


## 6. Identify Data Quality Issues


In [ ]:
# Identify multilabel fields
print("=" * 80)
print("6.1 Multilabel Fields")
print("=" * 80)
multilabel_cols = identify_multilabel_columns(df_computers)
print(f"Columns with multilabel data: {len(multilabel_cols)}")
print(f"\nFirst 10 multilabel columns:")
for col in multilabel_cols[:10]:
    sample = df_computers[col].dropna().head(2)
    if len(sample) > 0:
        print(f"\n{col}:")
        for val in sample:
            print(f"  - {val}")


In [ ]:
# Storage format issues
print("=" * 80)
print("6.2 Storage Format Issues")
print("=" * 80)
storage_col = 'Disco duro_Capacidad de memoria SSD'
if storage_col in df_computers.columns:
    sample = df_computers[storage_col].dropna().head(10)
    print(f"\nSample values from {storage_col}:")
    for val in sample:
        print(f"  - {val}")


In [ ]:
# Screen size issues
print("=" * 80)
print("6.3 Screen Size Issues")
print("=" * 80)
screen_col = 'Pantalla_Tamaño de la pantalla'
if screen_col in df_computers.columns:
    sample = df_computers[screen_col].dropna().head(10)
    print(f"\nSample values from {screen_col}:")
    for val in sample:
        print(f"  - {val}")


In [ ]:
# CPU/GPU naming issues
print("=" * 80)
print("6.4 CPU/GPU Naming Issues")
print("=" * 80)
cpu_col = 'Procesador_Procesador'
if cpu_col in df_computers.columns:
    sample = df_computers[cpu_col].dropna().head(10)
    print(f"\nSample CPU names:")
    for val in sample:
        print(f"  - {val}")


## 7. Data Cleaning

Now we'll apply all cleaning transformations step by step.


In [ ]:
# Start with a copy of the original dataframe
df_clean = df_computers.copy()
print("Starting cleaning process...")
print(f"Initial shape: {df_clean.shape}")


### 7.1 Clean Multilabel Fields


In [ ]:
# Clean multilabel fields
print("=" * 80)
print("7.1 Cleaning Multilabel Fields")
print("=" * 80)

# Preserve some columns if needed (like 'Ofertas' which might be informative)
preserve_cols = ['Ofertas']  # Add any columns you want to preserve

for col in multilabel_cols:
    if col not in preserve_cols:
        before_sample = df_clean[col].dropna().iloc[0] if len(df_clean[col].dropna()) > 0 else None
        df_clean[col] = clean_multilabel_series(df_clean[col])
        after_sample = df_clean[col].dropna().iloc[0] if len(df_clean[col].dropna()) > 0 else None
        if before_sample != after_sample and before_sample is not None:
            print(f"\n{col}:")
            print(f"  Before: '{before_sample}'")
            print(f"  After:  '{after_sample}'")

print(f"\nCleaned {len(multilabel_cols)} multilabel columns")


### 7.2 Clean Storage Fields


In [ ]:
# Clean storage fields
print("=" * 80)
print("7.2 Cleaning Storage Fields")
print("=" * 80)

config = get_default_config()
df_clean = clean_storage_fields(
    df_clean,
    ssd_col=config['storage']['ssd_col'],
    hdd_col=config['storage']['hdd_col'],
    storage_type_col=config['storage']['storage_type_col']
)

# Show examples
print("\nStorage cleaning examples:")
if 'storage_total_gb' in df_clean.columns:
    sample = df_clean[['Disco duro_Capacidad de memoria SSD', 'ssd_gb', 'storage_total_gb']].dropna().head(5)
    print(sample)


### 7.3 Clean Screen Fields


In [ ]:
# Clean screen fields
print("=" * 80)
print("7.3 Cleaning Screen Fields")
print("=" * 80)

df_clean = clean_screen_fields(
    df_clean,
    size_col=config['screen']['size_col'],
    resolution_col=config['screen']['resolution_col']
)

# Show examples
print("\nScreen cleaning examples:")
if 'screen_size_inches' in df_clean.columns:
    sample = df_clean[[config['screen']['size_col'], 'screen_size_inches']].dropna().head(5)
    print(sample)


### 7.4 Clean CPU/GPU Fields


In [ ]:
# Clean CPU/GPU fields
print("=" * 80)
print("7.4 Cleaning CPU/GPU Fields")
print("=" * 80)

df_clean = create_cleaned_cpu_gpu_columns(
    df_clean,
    cpu_col=config['cpu_gpu']['cpu_col'],
    gpu_col=config['cpu_gpu']['gpu_col']
)

# Show examples
if 'cpu_clean' in df_clean.columns:
    print("\nCPU cleaning examples:")
    sample = df_clean[[config['cpu_gpu']['cpu_col'], 'cpu_clean']].dropna().head(5)
    print(sample)


### 7.5 Extract Numeric Values from Other Fields


In [ ]:
# Extract numeric from other fields
print("=" * 80)
print("7.5 Extracting Numeric Values")
print("=" * 80)

numeric_fields = [
    ('RAM_Memoria RAM', 'ram_gb'),
    ('Procesador_Frecuencia de reloj', 'cpu_freq_ghz'),
    ('Medidas y peso_Peso', 'weight_kg'),
    ('Alimentación_Vatios-hora', 'battery_wh'),
]

for col, new_col in numeric_fields:
    if col in df_clean.columns:
        df_clean[new_col] = extract_numeric_series(df_clean[col])
        print(f"Created {new_col} from {col}")

# Show examples
print("\nNumeric extraction examples:")
if 'ram_gb' in df_clean.columns:
    sample = df_clean[['RAM_Memoria RAM', 'ram_gb']].dropna().head(5)
    print(sample)


### 7.6 Missing Value Imputation


In [ ]:
# Missing value imputation
print("=" * 80)
print("7.6 Missing Value Imputation")
print("=" * 80)

# Numeric columns: median imputation
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
imputed_numeric = 0
for col in numeric_cols:
    if df_clean[col].isna().any():
        median_val = df_clean[col].median()
        if pd.notna(median_val):
            missing_count = df_clean[col].isna().sum()
            df_clean[col].fillna(median_val, inplace=True)
            imputed_numeric += missing_count

print(f"Imputed {imputed_numeric} missing values in numeric columns using median")

# Categorical columns: mode imputation or "Unknown"
categorical_cols = df_clean.select_dtypes(include=['object']).columns
imputed_categorical = 0
for col in categorical_cols:
    if df_clean[col].isna().any():
        mode_val = df_clean[col].mode()
        if len(mode_val) > 0:
            missing_count = df_clean[col].isna().sum()
            df_clean[col].fillna(mode_val[0], inplace=True)
            imputed_categorical += missing_count
        else:
            missing_count = df_clean[col].isna().sum()
            df_clean[col].fillna('Unknown', inplace=True)
            imputed_categorical += missing_count

print(f"Imputed {imputed_categorical} missing values in categorical columns using mode/Unknown")

# Verify no missing values remain
remaining_missing = df_clean.isnull().sum().sum()
print(f"\nRemaining missing values: {remaining_missing}")


## 8. Visualizations


In [ ]:
# Distribution plots for key numeric fields
if 'storage_total_gb' in df_clean.columns:
    plt.figure(figsize=(12, 6))
    df_clean['storage_total_gb'].hist(bins=50, edgecolor='black')
    plt.title('Distribution of Total Storage (GB)', fontsize=14, fontweight='bold')
    plt.xlabel('Storage (GB)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../docs/storage_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
if 'screen_size_inches' in df_clean.columns:
    plt.figure(figsize=(12, 6))
    df_clean['screen_size_inches'].hist(bins=30, edgecolor='black')
    plt.title('Distribution of Screen Size (inches)', fontsize=14, fontweight='bold')
    plt.xlabel('Screen Size (inches)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../docs/screen_size_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
if 'ram_gb' in df_clean.columns:
    plt.figure(figsize=(12, 6))
    df_clean['ram_gb'].hist(bins=30, edgecolor='black')
    plt.title('Distribution of RAM (GB)', fontsize=14, fontweight='bold')
    plt.xlabel('RAM (GB)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../docs/ram_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
# Create output directory
os.makedirs('../data/clean', exist_ok=True)

# Export cleaned dataset
output_path = '../data/clean/db_computers_cleaned.csv'
df_clean.to_csv(output_path, index=False)

print("=" * 80)
print("CLEANED DATASET EXPORTED")
print("=" * 80)
print(f"\nOutput path: {output_path}")
print(f"Final shape: {df_clean.shape}")
print(f"Original shape: {df_computers.shape}")
print(f"\nNew columns created:")
new_cols = [col for col in df_clean.columns if col not in df_computers.columns]
for col in new_cols:
    print(f"  - {col}")


## 10. Summary Statistics

Final summary of the cleaned dataset.


In [ ]:
# Summary statistics for key numeric fields
print("=" * 80)
print("SUMMARY STATISTICS - CLEANED DATASET")
print("=" * 80)

summary_cols = ['storage_total_gb', 'ssd_gb', 'hdd_gb', 'screen_size_inches', 
                'ram_gb', 'cpu_freq_ghz', 'weight_kg', 'battery_wh']
available_cols = [col for col in summary_cols if col in df_clean.columns]

if available_cols:
    print("\nNumeric Summary:")
    print(df_clean[available_cols].describe())


In [ ]:
# Data quality summary
print("\n" + "=" * 80)
print("DATA QUALITY SUMMARY")
print("=" * 80)
print(f"\nOriginal dataset:")
print(f"  Rows: {df_computers.shape[0]}")
print(f"  Columns: {df_computers.shape[1]}")
print(f"  Missing values: {df_computers.isnull().sum().sum()}")

print(f"\nCleaned dataset:")
print(f"  Rows: {df_clean.shape[0]}")
print(f"  Columns: {df_clean.shape[1]}")
print(f"  Missing values: {df_clean.isnull().sum().sum()}")

print(f"\nCleaning operations completed:")
print(f"  ✓ Multilabel fields cleaned: {len(multilabel_cols)}")
print(f"  ✓ Storage fields standardized")
print(f"  ✓ Screen fields normalized")
print(f"  ✓ CPU/GPU fields cleaned for matching")
print(f"  ✓ Numeric values extracted")
print(f"  ✓ Missing values imputed")
